In [37]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast
pd.options.display.max_columns=None

In [38]:
attack=pd.read_csv('../ETL/datos/attacks_limpieza_completa.csv', index_col=0)
df=pd.read_csv('../ETL/datos/df_API_paises.csv', index_col=0)

In [39]:
class Concat_Paises():
    def __init__ (self,producto):
        self.producto=producto
        
    def paises(self):

        df_concat=pd.DataFrame()
        dict={'USA':[39.783730,-100.445882],'Australia':[-24.77610,134.755], 'South Africa':[-28.8166236,24.991639],\
            'New Zealand':[-41.5000831,172.8344077],'Papua New Guinea':[-5.6816069,144.2489081] }
            
        for k,v in dict.items():
                
            url = f'http://www.7timer.info/bin/api.pl?lon=-{v[1]}&lat={v[0]}&product={self.producto}&output=json'

            response = requests.get(url)
            df=pd.json_normalize(response.json()["dataseries"])
            df["latitud"] = v[0]
            df["longitud"] = v[1]
            df['paises']= k
            df_concat=pd.concat([df_concat,df], axis=0, ignore_index=True)

        return df_concat

In [40]:
class Limpiar_dfs():
    
    def __init__ (self,dataframe1,dataframe2):
        self.dataframe1=dataframe1
        self.dataframe2=dataframe2
        #self.df_left=df_left
        #self.df_rigth=df_rigth
    
    def limpiar_attack (self):
        df_attack=self.dataframe1[(self.dataframe1["country"]=="usa") | (self.dataframe1["country"]=="australia")|\
            (self.dataframe1["country"]=="south africa") | (self.dataframe1["country"]=="papua new guinea")|\
            (self.dataframe1["country"]=="new zealand")]
        df_attack.rename(columns={'country':'id_paises'}, inplace=True)
        df_final=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])
        df_final=df_final.reset_index()
        return df_final
    
    def limpieza_clima(self):
        #Limpiamos la columna de rh_profile primero a series y luego a columnas en el bbucle for
        self.dataframe2['rh_profile']=self.dataframe2['rh_profile'].apply(ast.literal_eval)
        rh = self.dataframe2['rh_profile'].apply(pd.Series)

        for i in range(len(rh.columns)):
            rh_col0=rh[i].apply(pd.Series)
            nombre="rh"+ rh_col0["layer"][115]
            valores= list(rh_col0["rh"])
            self.dataframe2.insert(i, nombre, valores)
        #Eliminamos la columna que ya no necesitamos
        self.dataframe2=self.dataframe2.drop(['rh_profile'], axis=1)
        
        #Hacemos lo mismo para la columnas wind_profile
        self.dataframe2['wind_profile']=self.dataframe2['wind_profile'].apply(ast.literal_eval)
        wind = self.dataframe2['wind_profile'].apply(pd.Series)

        for i in range(len(wind.columns)):
            wind_col0=wind[i].apply(pd.Series)
            nombre="wind_"+ wind_col0["layer"][1]
            valores= list(wind_col0["direction"])
            self.dataframe2.insert(i, nombre, valores)
            
        self.dataframe2=self.dataframe2.drop(['wind_profile'], axis=1)
        
        df2=self.dataframe2.groupby('paises').agg(['mean'])
        df2=df2.reset_index()
        df2.rename(columns={'paises':'id_paises'}, inplace=True)
        def minuscula(col):
            try:
                col=col.lower()
                return col
            except:
                pass
        df2["id_paises"]=df2["id_paises"].apply(minuscula)

        return df2
    
    

In [45]:
Limpieza=Limpiar_dfs(attack,df)


In [51]:
df_attack=Limpieza.limpiar_attack #No funciona :(

In [52]:
df_clima=Limpieza.limpieza_clima

In [53]:
df_clima

<bound method Limpiar_dfs.limpieza_clima of <__main__.Limpiar_dfs object at 0x7fd560304d00>>

In [44]:
def merge(self): #Esta función está persanda para hacer un left. Para que se tenga en cuenta
    merge_final=pd.merge(left=self.df_left, right=self.df_rigth, how='left', left_on='id_paises', right_on='id_paises')
    return merge_final